In [1]:
import pandas as pd
from nltk import edit_distance
import re


In [2]:
trends = pd.read_csv('Datasets/aggregated/preproccesed_trends_data.csv')
trends.Region = trends.Region.apply(lambda x: 'Region ' + x)

mun = pd.read_csv('helpers/Regions_and_Municipalities/Municipalities.csv', skiprows= 1, names = ['id', 'Municipality', 'City'])



reg_mun = pd.read_csv('helpers/Regions_and_Municipalities/Regions_and_municipalities.csv', skiprows = 4, names = ['area', 'pop'], encoding='cp1252')

reg_args = ~reg_mun.area.apply(lambda x: re.search(pattern= 'Region', string= x)).isna()
reg_args = reg_args[reg_args == True]

reg_dict = {reg_mun.area[idx] : [reg_mun.area[idx+1 : reg_args.index[enum+1] ].values ] if enum <= 3 else [reg_mun.area[idx+1 : ].values] for enum, idx in enumerate(reg_args.index)  }

reg_df = pd.DataFrame.from_dict(reg_dict,orient='index').rename_axis('Region').reset_index()
reg_df = reg_df.explode(0).rename(columns={0 : 'Municipality'})
reg_df.Municipality = reg_df.Municipality.apply(lambda x: x + ' Kommune')
reg_df.Municipality[reg_df.Municipality == 'København Kommune'] = 'Københavns Kommune'


mun = mun.merge(reg_df, on = 'Municipality', how = 'left')

mun = mun[~mun.Region.isna()]

group_df = mun.groupby(['City', 'Region'], as_index= False).count().reset_index()
group_df = group_df[['City', 'Region']][group_df.Municipality > 1]
group_df = group_df[~group_df.City.isin(['Uden fast bopæl', 'Landdistrikter'])]

group_df.to_csv('helpers/cities_in_multiple_municipalities.csv')


In [3]:
group_df = pd.read_csv('helpers/cities_in_multiple_municipalities_done.csv', sep =';')

group_df = group_df[~group_df.City.isin(group_df.City[group_df.City.duplicated()])]


for i in range(len(group_df.City)):
    idx = ~mun.City.apply(lambda x: re.search(pattern= group_df.DST_name.iloc[i], string= x)).isna()
    mun.City[idx] = group_df.City.iloc[i]

for i in range(len(group_df.City)):
    idx = ~mun.City.apply(lambda x: re.search(pattern= group_df.City.iloc[i], string= x)).isna()
    idx_reg = ~mun.Region.apply(lambda x: re.search(pattern= group_df.Region.iloc[i], string= x)).isna()
    mun.Municipality[(idx & idx_reg)] = group_df.Municipality.iloc[i]

mun = mun[['Municipality', 'City', 'Region']].drop_duplicates()

test = trends.merge(mun[['City', 'Region', 'Municipality']], on=['City', 'Region'], how = 'left')

/tmp/ipykernel_17334/4007665638.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mun.City[idx] = group_df.City.iloc[i]
/tmp/ipykernel_17334/4007665638.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mun.Municipality[(idx & idx_reg)] = group_df.Municipality.iloc[i]


In [4]:
def get_similar_city(city, cities):
    distances = cities.apply(edit_distance, s2 = city)
    distances = distances[distances <= 2]
    return cities.values[distances.index]

city_ser = pd.Series(test.City[test.Municipality.isna()].unique())

new = city_ser.apply(get_similar_city, cities = pd.Series(mun.City.unique()))
city_frame = pd.DataFrame({'City' : city_ser, 'Candidates' : new})



In [5]:
def correct_city(city_frame, muns, full):
    muns = muns[muns.City.isin(city_frame.Candidates)]
    full_reg = full.Region[full.City == city_frame.City].unique()
    muns = muns[muns.Region.isin(full_reg)]
    muns['old'] = city_frame.City
    return muns

candidate_df = pd.DataFrame()
for i in city_frame.index:
    candidate_df = candidate_df.append(correct_city(city_frame.iloc[i], mun, test))

candidate_df.to_csv('helpers/cities_for_manual_inspection.csv')

def is_in(candidate, muns):
    return ~muns.old.isin([candidate]).any()



no_DST = city_ser[city_ser.apply(is_in, muns = candidate_df)]

We do manual inspection

In [6]:
candidate_df = pd.read_csv('helpers/cities_for_manual_inspection_done.csv', sep = ';')

replacements = candidate_df[candidate_df.annotations == 1]

no_DST = no_DST.append(candidate_df.old[candidate_df.annotations == 0])

no_DST.to_csv('helpers/cities_missing_municipality.csv')

We do manual inspection

In [7]:
no_DST = pd.read_csv('helpers/cities_missing_municipality_done.csv', sep = ';', skiprows=1, names = ['id', 'DST_name', 'City', 'Municipality'])
no_DST = no_DST[['DST_name', 'City', 'Municipality']].drop_duplicates()

In [8]:

for i in range(len(replacements.old)):
    idx = ~test.City.apply(lambda x: re.search(pattern= replacements.old.iloc[i], string= x)).isna()
    test.City[idx] = replacements.City.iloc[i]
    test.Municipality[idx] = replacements.Municipality.iloc[i]

for i in range(len(no_DST.City)):
    idx = ~test.City.apply(lambda x: re.search(pattern= no_DST.City.iloc[i], string= x)).isna()
    test.City[idx] = no_DST.DST_name.iloc[i]

for i in range(len(no_DST.City)):
    idx = ~test.City.apply(lambda x: re.search(pattern= no_DST.DST_name.iloc[i], string= x)).isna()
    test.Municipality[idx] = no_DST.Municipality.iloc[i]

test.Municipality[test.City == 'Ballerup'] = 'Ballerup Kommune'

/tmp/ipykernel_17334/3173607246.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.City[idx] = replacements.City.iloc[i]
/tmp/ipykernel_17334/3173607246.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.Municipality[idx] = replacements.Municipality.iloc[i]
/tmp/ipykernel_17334/3173607246.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.City[idx] = no_DST.DST_name.iloc[i]
/tmp/ipykernel_17334/3173607246.py:12: Setti

In [9]:
FB_estimates = pd.read_csv('Datasets/aggregated/FB_estimates.csv')

def string_split(series):
    return series.split(',')[0]

FB_estimates.City = FB_estimates.City.apply(string_split)
FB_estimates.Region[FB_estimates.Region == 'North Denmark Region'] = 'Region Nordjylland'
FB_estimates.Region[FB_estimates.Region == 'Region of Southern Denmark'] = 'Region Syddanmark'
FB_estimates.Region[FB_estimates.Region == 'Capital Region of Denmark'] = 'Region Hovedstaden'
FB_estimates.Region[FB_estimates.Region == 'Central Denmark Region'] = 'Region Midtjylland'
FB_estimates.Region[FB_estimates.Region == 'Zealand Region'] = 'Region Sjælland'




FB_df = FB_estimates.merge(mun[['City', 'Region', 'Municipality']], on=['City', 'Region'], how = 'left')

for i in range(len(replacements.old)):
    idx = ~FB_df.City.apply(lambda x: re.search(pattern= replacements.old.iloc[i], string= x)).isna()
    FB_df.City[idx] = replacements.City.iloc[i]
    FB_df.Municipality[idx] = replacements.Municipality.iloc[i]


for i in range(len(no_DST.City)):
    idx = ~FB_df.City.apply(lambda x: re.search(pattern= no_DST.City.iloc[i], string= x)).isna()
    FB_df.Municipality[idx] = no_DST.Municipality.iloc[i]

FB_df['Upper bound'][(FB_df.City == 'Herlev')] = 4100
FB_df['Lower bound'][(FB_df.City == 'Herlev')] = 3500


FB_df = FB_df[~(FB_df.City == 'Herlev Kommune')]

FB_isna = FB_df[FB_df.Municipality.isna()]


FB_isna.to_csv('helpers/FB_with_no_Municipality.csv')


/tmp/ipykernel_17334/587513556.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FB_estimates.Region[FB_estimates.Region == 'North Denmark Region'] = 'Region Nordjylland'
/tmp/ipykernel_17334/587513556.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FB_estimates.Region[FB_estimates.Region == 'Region of Southern Denmark'] = 'Region Syddanmark'
/tmp/ipykernel_17334/587513556.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FB_es

In [10]:
FB_isna = pd.read_csv('helpers/FB_with_no_Municipality_done.csv', sep = ';')


for i in range(len(FB_isna.City)):
    idx = ~FB_df.City.apply(lambda x: re.search(pattern= FB_isna.City.iloc[i], string= x)).isna()
    FB_df.City[idx] = FB_isna.DST_name.iloc[i]


for i in range(len(FB_isna.City)):
    idx = ~FB_df.City.apply(lambda x: re.search(pattern= FB_isna.DST_name.iloc[i], string= x)).isna()
    idx_reg = ~FB_df.Region.apply(lambda x: re.search(pattern= FB_isna.Region.iloc[i], string= x)).isna()
    FB_df.Municipality[(idx & idx_reg)] = FB_isna.Municipality.iloc[i]

FB_df = FB_df[~FB_df.Municipality.isna()]

FB_df['year'] = 2021

/tmp/ipykernel_17334/2556284496.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FB_df.City[idx] = FB_isna.DST_name.iloc[i]
/tmp/ipykernel_17334/2556284496.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FB_df.Municipality[(idx & idx_reg)] = FB_isna.Municipality.iloc[i]


In [11]:
ev_df = pd.read_csv('Datasets/aggregated/ev_population.csv', encoding ='cp1252', skiprows=1, names = ['id', 'Municipality', 'year', 'EV_pop'])

ev_df.Municipality = ev_df.Municipality.apply(lambda x: x + ' Kommune')
ev_df.Municipality[ev_df.Municipality == 'København Kommune'] = 'Københavns Kommune'


merged_df = FB_df.merge(test, on = ['City', 'Region', 'Municipality', 'year'], how='outer')

merged_df = merged_df.merge(ev_df, on = ['Municipality', 'year'], how = 'left')

/tmp/ipykernel_17334/366610482.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ev_df.Municipality[ev_df.Municipality == 'København Kommune'] = 'Københavns Kommune'


In [12]:
dst_population = pd.read_csv('Datasets/aggregated/population_stats.csv', encoding='cp1252', skiprows=1, names = ['Area_id', 'Area_name', 'year', 'pop', 'pop_dens'])

def string_split(series, pattern):
    return series.split(pattern)[0]

def get_mun(rows):
    rows['Municipality'] = rows.Area_name.iloc[0]
    return rows['Municipality']

dst_population.Area_name = dst_population.Area_name.apply(string_split, pattern = ' (')
dst_population['Mun_code'] = dst_population.Area_id.apply(string_split, pattern = '-')


mun_map = dst_population.groupby('Mun_code', as_index = False).apply(get_mun).reset_index()
dst_population['Municipality'] = ''

dst_population['Municipality'][mun_map.level_1] = mun_map.Municipality

dst_population = dst_population[~(dst_population.Area_name == dst_population.Municipality)]
dst_population = dst_population.rename(columns = {'Area_name' : 'City'})

merged_df = merged_df.merge(dst_population, on = ['City', 'Municipality', 'year'], how = 'left')


/tmp/ipykernel_17334/1182527733.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dst_population['Municipality'][mun_map.level_1] = mun_map.Municipality


In [13]:
charger_df = pd.read_csv('Datasets/aggregated/muni_aggr_charging_data.csv', )
charger_df = charger_df.rename(columns={'up_until_year' : 'year', 'municipality' : 'Municipality'})
charger_df.Municipality[charger_df.Municipality == 'København'] = 'Københavns'
charger_df.Municipality[charger_df.Municipality == 'Høje Taastrup'] = 'Høje-Taastrup'
charger_df.Municipality = charger_df.Municipality.apply(lambda x: x + ' Kommune')

merged_df = merged_df.merge(charger_df, on = ['Municipality', 'year'], how = 'left')

/tmp/ipykernel_17334/629087313.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charger_df.Municipality[charger_df.Municipality == 'København'] = 'Københavns'
/tmp/ipykernel_17334/629087313.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charger_df.Municipality[charger_df.Municipality == 'Høje Taastrup'] = 'Høje-Taastrup'


In [14]:
old_colnames = ['City', 'Municipality', 'Region', 'year', 'Lower bound', 'Upper bound', 'slope', 'intercept', 'R2', 'R2_adj', 'mean', 'trending', \
 'EV_pop', 'pop', 'pop_dens', 'count_of_stations', 'plugs', 'closest_in_meters', 'closest_in_meters_std']
new_colnames = ['City', 'Municipality', 'Region', 'year', 'FB_audience_size_lower_bound', 'FB_audience_size_upper_bound', 'Trends_slope', 'Trends_intercept', \
    'Trends_regression_R2', 'Trends_regression_R2_adj', 'Trends_mean', 'Trends_is_trending', 'DST_EV_population', 'DST_population_size', 'DST_population_density', \
    'Plugshare_count_of_stations', 'Plugshare_count_of_plugs', 'Plugshare_closest_in_meters', 'Plugshare_closest_in_meters_std']

merged_df = merged_df[old_colnames]
merged_df.columns = new_colnames

merged_df.to_csv('EVDK.csv')


In [15]:
inspect_df = merged_df[~merged_df['Trends_slope'].isna()]
inspect_df = inspect_df[~inspect_df['DST_population_size'].isna()]
inspect_df = inspect_df[~(inspect_df.year.isin([2021]) & inspect_df['FB_audience_size_upper_bound'].isna())]

data21 = inspect_df[(inspect_df.year == 2021) & (inspect_df['FB_audience_size_upper_bound'] != 1000)]

data21.to_csv('EVDK_2021.csv')
